# **NLP_Tutorial_03_Word_Embedding_2**

-> word2vec 이후 감정분석에 적용하기

### **Package 설치**
 - datasets (huggingface 자연어처리 dataset)
 - konlpy (한글 자연어처리 Tool)
 - tqdm (반복문의 Progress Bar)


In [ ]:
!pip install datasets
!pip install konlpy
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.6 MB/s 
     |████████████████████████████████| 182 kB 30.7 MB/s 
     |████████████████████████████████| 132 kB 43.0 MB/s 
     |████████████████████████████████| 212 kB 41.7 MB/s 
     |████████████████████████████████| 127 kB 20.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 29.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **3-0. 데이터셋 준비**
Naver Sentiment Movie Corpus(nsmc) <br>
영화 리뷰를 별점을 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [ ]:
from datasets import load_dataset

# nsmc 데이터셋 다운로드
dataset = load_dataset("nsmc")

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
dataset["train"]

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [ ]:
dataset["train"][0]

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

In [ ]:
dataset["train"][0]["document"]

'아 더빙.. 진짜 짜증나네요 목소리'

```
데이터 셋에서 데이터와 라벨을 리스트에 저장한다.
```

In [ ]:
train_sents = [dataset["train"][idx]["document"] for idx in range(len(dataset["train"]))]
train_labels = [dataset["train"][idx]["label"] for idx in range(len(dataset["train"]))]
test_sents = [dataset["test"][idx]["document"] for idx in range(len(dataset["test"]))]
test_labels = [dataset["test"][idx]["label"] for idx in range(len(dataset["test"]))]

```
지난 시간에는 1000개의 문장을 보정, 정출했기에 라이브러리를 사용했다.

이번에는 150000개를 다 사용할 것이다. 데이터의 수가 많으면 라이브러리 사용에 시간이 많이 걸린 다는 것을 유의하고 강의에서는 시간관계상 띄어쓰기나 맞춤법 등 보정은 안 하고 그 외에 정제 및 추출만 하겠다.

1. 불용어
2. 한글이 아닌 글자 제거
3. 어간 추출


-> 순서
토큰화 > 정제 및 추출 > 인코딩이다. 근데 okt에는 토큰화가 아닌 원본 문장을 넣어야하고 그러면 토큰화 결과를 반환한다. 

+ 보정은 토큰화 전에 하는데 시간이 많이 걸려서 패스한다.
```

In [ ]:
# 원본 문장을 넣으면 토큰화 결과 반환
for sent_i in tqdm(train_sents[:5]):
  tokens = okt.morphs(sent_i, stem=True) # 어간 추출
  print(sent_i)
  print(tokens)

100%|██████████| 5/5 [00:00<00:00, 147.45it/s]

아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증나다', '목소리']
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍다', '않다']
너무재밓었다그래서보는것을추천한다
['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한', '다']
교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['교도소', '이야기', '구먼', '..', '솔직하다', '재미', '는', '없다', '..', '평점', '조정']
사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스럽다', '연기', '가', '돋보이다', '영화', '!', '스파이더맨', '에서', '늙다', '보이다', '하다', '커스틴', '던스트', '가', '너무나도', '이쁘다', '보이다']


In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm

# stopwords 정의
stopwords = ["하다","한","에","와","자","과","걍","잘","좀","는","의","가","이","은","들"]

okt = Okt()

train_nsmc_corpus = []
for sent_i in tqdm(train_sents):
  
  
  # 한글이 아닌 글자는 제거
  # 정규식으로 한글을 뜻하는 re이다. 그것을 "" 아무것도 아닌 것으로 대체한다. = 제거힌디.
  new_sent_i = sent_i.replace("[^가-힣ㄱ-하-ㅣ ]","") 



  tokens = okt.morphs(new_sent_i, stem=True) # 한글 아닌 글자 제거한 후 어간 추출
  tokens = [word_i for word_i in tokens if word_i not in stopwords] # 불용어 제거

  train_nsmc_corpus.append(tokens) # 결과 : 한글로 된 어간이 추출된 단어로만 구성된다.


test_nsmc_corpus = []
for sent_i in tqdm(test_sents):
  new_sent_i = sent_i.replace("[^가-힣ㄱ-하-ㅣ ]","")
  tokens = okt.morphs(new_sent_i, stem=True)
  tokens = [word_i for word_i in tokens if word_i not in stopwords]

  test_nsmc_corpus.append(tokens)

100%|██████████| 50000/50000 [03:14<00:00, 257.68it/s]


```
bow, tfidf는 단어 임베딩을 하기 위해서 vocab_size만한 벡터가 필요하고 대부분이 0인 sparse한 벡터였다.(의미가 없는 데이터)

이번에는 dense한 밀집 벡터 형식의 단어 임베딩을 해보자

+ 또한 bow, tfidf는 단어 임베딩이라고 하지만 단어의 출현 빈도를 보기 때문에 문장 단위로 수치 표현을 하는 것이었다. 이번엔 단어 하나만을 숫자로 표현해보자
```

### **3-1. Word2Vec (CBoW)**
Word2Vec은 이웃 단어 예측을 통해 단어의 임베딩을 학습하는 기법 <br>
CBoW 기법 / Skip-Gram 기법 <br> <br> 
CBoW는 주위(Window) 단어를 보고 중심 단어를 예측

```
ex) 5개의 단어를 본다면 앞에 두개와 뒤에 두개를 보고 3번 째 단어가 어떤 단어가 되어야하는 지를 맞추도록 학습이 된다.

이웃한 단어 사이의 관계를 반영하는 임베딩을 만들 수 있다.
```

```
인자
1. sentences : 데이터
2. size : 임베딩 차원
3. window : 윈도우
4. min_count : 빈도수가 5개 이하인 단어는 임베딩하지 않겠다
5. workers : 프로세스 개수(= cpu개수 몇개로 학습할 건가)
6. sg = cbow/ skipgram
```

In [ ]:
from gensim.models import Word2Vec

'''
# 나는 학교에 지하철을 타고 간다
# 나는 학교에 [      ] 타고 간다 -> 윈도우의 주변 단어로 중심단어를 맞추도록 학습이 된다.
# 차, 버스, 지하철, 자전거 -> 그러면 모델은 가능한 대중 교통을 다 생각할 수 있고 이렇게 가능한 것들을 다 예측하도록 학습이 된다.
# 이렇게 되면 차, 버스, 지하철, 자전거 등의 대중교통 들은 비슷한 단어 임베딩을 가지게 된다!!!
# 이때 예측과 정답의 차이로 학습을 한다. -> 따라서 단어들간의 관계가 반영된다.
'''

CBOW_model = Word2Vec(sentences=train_nsmc_corpus , size = 100 , window = 5, min_count=5, workers = 4, sg = 0)
# sg = 0 (CBOW)  /  sg = 1 (skip gram)

In [ ]:
# 송강호라는 벡터 구하기 -> 100-dim으로 구성
CBOW_model.wv["송강호"]

array([-0.2628437 , -0.4772478 ,  0.23714902,  0.108261  , -0.22606453,
        0.45416036,  0.3946831 ,  0.14584689,  0.29550198, -0.3781312 ,
        0.05215524, -0.03210475, -0.21495825,  0.19717748, -0.12327231,
        0.11480867,  0.22195497, -0.12611334, -0.05030181, -0.11514618,
       -0.2236208 ,  0.14717443,  0.07398075, -0.11808933,  0.2227676 ,
       -0.065589  , -0.19093974,  0.28443652, -0.10423951, -0.6661239 ,
        0.11916403, -0.351201  , -0.22943462,  0.01228224,  0.02278129,
        0.06770418,  0.16982408, -0.18317719,  0.15024005,  0.02706133,
        0.25394517,  0.01896458,  0.367126  ,  0.07914708, -0.05828024,
       -0.26531145, -0.16744003,  0.43909192,  0.36344227, -0.4778765 ,
       -0.12529951,  0.01365054,  0.02586485, -0.07423191,  0.04482049,
        0.07650961, -0.205869  , -0.3518455 ,  0.2614185 ,  0.05062447,
        0.47692335, -0.28031155,  0.20891698, -0.42138496,  0.11372356,
        0.32726842,  0.07744976,  0.15001541, -0.3298002 ,  0.02

```
중간에 들어올 단어가 무엇일지 예측을 하면서 학습이 되어서 예측되는 단어들이 비슷한 벡터가 될것이라고 했다.

결과 비슷한 단어들은 비슷한 상황에서의 단어들이 나온다.


```


In [ ]:
# 이번 영화 ㅁ 연기가 진짜 좋았어라면 
# ㅁ에는 송강호, 한석규, 차승원, 황정민, 이정재가 들어갈 수 있었던 것이다.
CBOW_model.wv.most_similar("송강호", topn=5)

[('한석규', 0.9075385332107544),
 ('차승원', 0.8906582593917847),
 ('황정민', 0.887117326259613),
 ('이정재', 0.8853720426559448),
 ('신들리다', 0.8847559094429016)]

In [ ]:
# 사전에 없는 단어는 오류가 난다. lookup table을 만드는 것이기 때문이다. 따라서 예외처리를 해야한다.
CBOW_model.wv.most_similar("자연어", topn=5)

KeyError: ignored

In [ ]:
train_nsmc_corpus[:10]

[['아', '더빙', '..', '진짜', '짜증나다', '목소리'],
 ['흠',
  '...',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '....',
  '오버',
  '연기',
  '조차',
  '가볍다',
  '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '..', '솔직하다', '재미', '없다', '..', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '익살스럽다',
  '연기',
  '돋보이다',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙다',
  '보이다',
  '커스틴',
  '던스트',
  '너무나도',
  '이쁘다',
  '보이다'],
 ['막',
  '걸음',
  '마',
  '떼다',
  '3',
  '세',
  '부터',
  '초등학교',
  '1',
  '학년',
  '생인',
  '8',
  '살다',
  '영화',
  '.',
  'ㅋㅋㅋ',
  '...',
  '별',
  '반개',
  '도',
  '아깝다',
  '움',
  '.'],
 ['원작', '긴장감', '을', '제대로', '살리다', '.'],
 ['별',
  '반개',
  '도',
  '아깝다',
  '욕',
  '나오다',
  '이응경',
  '길용우',
  '연',
  '기',
  '생활',
  '몇',
  '년',
  '인지',
  '..',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫다',
  '납치',
  '.',
  '감금',
  '만',
  '반복',
  '반복',
  '..',
  '드라마',
  '가족',
  '도',
  '없다',
  '연기',
  '못',
  '사람',
  '만',
  '모',
  '엿',
  '네'],
 ['액션', '없다', '재미', '있다', '몇', '안되다', '영화'],
 ['왜케',
  '평

In [ ]:
len(train_nsmc_corpus)

150000

```
결과 : 지금 문장 150000개가 있다. 그 문장에 토큰화 > 정제 및 추출을 했다. 1500000개의 문장이 리스트에 한 문장씩 구분되어 있다. 

그러한 문장을 for문으로 한 문장 씩 불러서 각 단어를 임베딩하고 sent_emb으로 붙이고 전체 문장 배열에 append한다.

+ 추가로 문장의 단어를 임베딩한 후 문장의 임베딩은 단어 임베딩의 평균으로 한다.
```

In [ ]:
dim = 100
sent_emb = np.zeros(dim)

for i in train_nsmc_corpus[:10]:
    word_emb = CBOW_model.wv[i]
    sent_emb += word_emb
    print(sent_emb)

[-0.24741404  0.67173541  0.18896787  0.28265417 -0.01478071  0.5038864
 -1.08722007  0.49759418 -1.30356431  0.17740911  0.24749696  0.86684275
 -0.56336212 -0.13190328 -1.63394487 -0.30320615  1.33291435  0.38028178
 -0.03128756  1.34461188  0.65803874 -1.06445873 -0.58787864 -0.74107093
 -0.25719467  1.96874869  0.85347611 -0.26399794 -0.35309201  0.90471745
  0.45554194 -1.12287235 -0.51878393 -2.79717493  0.43166605  0.99555719
  1.30714214 -0.86195153  0.1531385   0.38983169  0.97367722 -0.501921
  1.60702813 -1.30994391  1.41272604  0.23485899 -0.74714935 -0.30797896
 -0.26043099 -0.02807765 -0.41553783 -0.92578727  0.77919459  1.30872345
 -0.88941544  0.1587359  -0.57749742 -0.69296068  0.69670159  1.10503042
  0.60368574  1.70543659 -1.20990086 -1.46070361 -0.17231928  0.33181959
 -0.27768391  1.31489861 -0.96715683  1.53960609  1.07003999  0.20555256
  0.80984432  1.31233513  0.71575469  1.03613687  1.05304503  0.60894489
  0.33415979  0.33395356 -0.01946349  0.20747575  0.04

KeyError: ignored

In [ ]:
import numpy as np

def sentence_embedding_word2vec(model, sentence, dim=100):
  sent_emb = np.zeros(dim)
  n = 0

  for word_i in sentence:
    # Vocab에 존재하지 않는 단어의 경우 예외처리
    try:
      word_emb = model.wv[word_i]
      sent_emb += word_emb
      n += 1
    except:
      pass
  
  # 문장 임베딩 = 단어 임베딩의 평균
  if n != 0:
    sent_emb = sent_emb / n

  return sent_emb

train_x_cbow = list()
test_x_cbow = list()

for train_sent in tqdm(train_nsmc_corpus):
  train_x_cbow.append(sentence_embedding_word2vec(CBOW_model, train_sent))
  
for test_sent in tqdm(test_nsmc_corpus):
  test_x_cbow.append(sentence_embedding_word2vec(CBOW_model, test_sent))


100%|██████████| 50000/50000 [00:02<00:00, 16712.70it/s]


In [ ]:
print(len(train_x_cbow))

150000


```
문장 임베딩 결과로 한 문장을 이루는 단어들의 임베딩의 평균을 사용한다.

예를 들어 한 문장이 3개의 단어가 5차원으로 임베딩이 되어있다면 [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]] 이렇게 되어있고 

평균은 [(1+1+1)/3, (2+2+2)/3, (3+3+3)/3, (4+4+4)/3, (5+5+5)/3] 이렇게 구할 것이다.
```

In [ ]:
print(train_x_cbow[0])

[-1.00505839 -0.81070619 -0.03293103 -0.52566174 -0.68931316  0.36177054
 -0.9444053  -0.19524381  0.55070054 -0.8609568   0.24253886 -0.36426238
 -1.16387616  0.10458526 -0.19977205  0.71803789  0.24559521 -0.23682014
  0.25998896 -0.92308732 -0.31607508 -0.07139069  0.44520867 -0.12184798
  0.08935051  0.74709111  0.43732359  0.02812632 -0.16734393 -0.52544974
 -0.56947294 -0.49256795  0.72977804 -0.08544813 -0.42400067  0.26071075
 -0.15846123 -0.47120438  0.3228762   0.85662265  0.35570175  0.07764144
 -1.17346953  0.17594641 -0.2289175   0.06001386 -0.52326286 -0.26134688
  0.20611642  0.41414792 -0.97819123 -0.17363668 -0.61534542 -0.34313061
  0.8130869  -0.48619541 -0.64887925  0.22245152 -0.27086574 -0.21967555
  0.20752169 -0.00119175  0.57154278 -0.0804174   0.29208229 -0.16091868
 -0.14943271 -0.18036344 -0.6067974  -0.10508175 -0.0694753  -0.54693572
 -0.46157151 -0.18619849 -0.0381568   0.89438328 -0.06924705  0.12277227
 -0.21866433 -0.18549544 -0.25335103 -0.06380832 -0

### **LogisticRegression** <br>
Linear Regression: 실수 범위의 값으로 예측 <br>
Logistic Regression: 0과 1 사이의 값으로 예측

-> 문장 임베딩 값이 수치값이니 w*x+b에 x 값으로 문장 임베딩 요소 값으로 넣어서 0과 1을 예측하자

In [ ]:
from sklearn.linear_model import LogisticRegression
linear_model_cbow = LogisticRegression()
linear_model_cbow.fit(train_x_cbow, train_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### 모델 성능

In [ ]:
train_pred = linear_model_cbow.predict(train_x_cbow)
# train_pred = [0, 1, 1, 0, ... ] => 0과 1로 이루어진 15만개의 예측 결과 리스트
# train_labels = [0, 0, 1, 1, ...] => 0과 1로 이루어진 15만개의 정답 리스트
print("train acc: {0}".format(np.mean(train_pred == train_labels))) # 맞는 것들의 평균이 정확도이다.

test_pred = linear_model_cbow.predict(test_x_cbow)
print("test acc: {0}".format(np.mean(test_pred == test_labels)))

train acc: 0.8096933333333334
test acc: 0.806


### **3-2. Word2Vec (Skip-Gram)**
Skip-Gram은 중심 단어를 보고 주위(Window) 단어들을 예측


In [ ]:
SkipGram_model = Word2Vec(sentences=train_nsmc_corpus , size = 100 , window = 5, min_count=5 , workers = 4 , sg = 1)

train_x_skipgram = list()
test_x_skipgram = list()

for train_sent in tqdm(train_nsmc_corpus):
  train_x_skipgram.append(sentence_embedding_word2vec(SkipGram_model, train_sent))
  
for test_sent in tqdm(test_nsmc_corpus):
  test_x_skipgram.append(sentence_embedding_word2vec(SkipGram_model, test_sent))

SkipGram_model.wv.most_similar("송강호", topn=5)

100%|██████████| 50000/50000 [00:02<00:00, 19624.89it/s]


[('안성기', 0.8696361780166626),
 ('황정민', 0.8552669286727905),
 ('한석규', 0.8440371155738831),
 ('윤계상', 0.8416322469711304),
 ('류덕환', 0.8358038663864136)]

In [ ]:
linear_model_skipgram = LogisticRegression()
linear_model_skipgram.fit(train_x_skipgram, train_labels)

LogisticRegression()

In [ ]:
train_pred = linear_model_skipgram.predict(train_x_skipgram)
print("train acc: {0}".format(np.mean(train_pred == train_labels)))
test_pred = linear_model_skipgram.predict(test_x_skipgram)
print("test acc: {0}".format(np.mean(test_pred == test_labels)))

train acc: 0.8260133333333334
test acc: 0.82134


### **3-3. GloVe**
Word2Vec은 이웃한 단어의 관계만 반영. 전체 코퍼스의 정보 반영 X <br>
GloVe는 이웃한 단어의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 학습 (전체 코퍼스의 정보 반영O)


In [ ]:
pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 974 kB 5.0 MB/s 


In [ ]:
from glove import Corpus, Glove

glove_corpus = Corpus()
glove_corpus.fit(train_nsmc_corpus, window=5)

glove_model = Glove(no_components=100, learning_rate=0.05)

glove_model.fit(glove_corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove_model.add_dictionary(glove_corpus.dictionary)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [ ]:
glove_model.most_similar("송강호")

[('김혜수', 0.8820887800824811),
 ('전도연', 0.8692602691476562),
 ('한석규', 0.868943840694518),
 ('황정민', 0.8219953944317955)]

In [ ]:
print(glove_model.dictionary["송강호"])

1571


In [ ]:
glove_model.word_vectors[glove_model.dictionary["송강호"]]

array([ 0.01816078, -0.06439144, -0.00099349,  0.03450806,  0.18384156,
        0.29463263, -0.05815017, -0.0278993 , -0.04618377,  0.03156437,
       -0.08516555,  0.05503753,  0.00833572,  0.19264104, -0.00294195,
       -0.0155024 ,  0.04933996, -0.0542288 , -0.06119193,  0.19291067,
       -0.05006303, -0.05321567,  0.26649126,  0.06193088,  0.06645491,
       -0.03550653, -0.05063277,  0.13164599,  0.00806965,  0.00775278,
       -0.13291446, -0.00402325,  0.09238119,  0.0215948 ,  0.08870745,
        0.07999845,  0.17685755, -0.0483145 ,  0.08885426, -0.00470385,
        0.00487702,  0.0150224 ,  0.2643144 ,  0.14368683,  0.00036187,
        0.06050917, -0.04313718, -0.01014807, -0.06239805,  0.0545082 ,
       -0.01298096, -0.09224155, -0.07451784,  0.04615526, -0.04694218,
        0.01253877,  0.07027624, -0.02808629,  0.02376414, -0.00198097,
        0.04510739,  0.03360412, -0.00560654,  0.15721051, -0.04011379,
       -0.0115198 , -0.00078922, -0.00724472, -0.06214793, -0.10

In [ ]:
def sentence_embedding_glove(model, sentence, dim=100):
  sent_emb = np.zeros(100)
  n = 0

  for word_i in sentence:
    try:
      word_emb = model.word_vectors[model.dictionary[word_i]]
      sent_emb += word_emb
      n += 1
    except:
      pass
  
  if n != 0:
    sent_emb = sent_emb / n

  return sent_emb

train_x_glove = list()
test_x_glove = list()

for train_sent in tqdm(train_nsmc_corpus):
  train_x_glove.append(sentence_embedding_glove(glove_model, train_sent))
  
for test_sent in tqdm(test_nsmc_corpus):
  test_x_glove.append(sentence_embedding_glove(glove_model, test_sent))

100%|██████████| 50000/50000 [00:01<00:00, 37024.04it/s]


In [ ]:
linear_model_glove = LogisticRegression()
linear_model_glove.fit(train_x_glove, train_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
train_pred = linear_model_glove.predict(train_x_glove)
print("train acc: {0}".format(np.mean(train_pred == train_labels)))
test_pred = linear_model_glove.predict(test_x_glove)
print("test acc: {0}".format(np.mean(test_pred == test_labels)))

train acc: 0.78466
test acc: 0.77944


### **3-4. 학습한 모델 저장 및 불러오기**


In [ ]:
# 저장
CBOW_model.save("word2vec_cbow.model")

# 불러오기
CBOW_model_2 = Word2Vec.load("word2vec_cbow.model")

cbow_vector_1 = CBOW_model.wv["송강호"]
cbow_vector_2 = CBOW_model_2.wv["송강호"]

if np.sum(np.abs(cbow_vector_1 - cbow_vector_2)) == 0:
  print("두 단어 임베딩이 일치함!")

In [ ]:
# 저장
glove_model.save("glove.model")

# 불러오기
glove_model_2 = glove_model.load("glove.model")

glove_vector_1 = glove_model.word_vectors[glove_model.dictionary["송강호"]]
glove_vector_2 = glove_model_2.word_vectors[glove_model.dictionary["송강호"]]

if np.sum(np.abs(glove_vector_1 - glove_vector_2)) == 0:
  print("두 단어 임베딩이 일치함!")